In [1]:
# deal with 344_zf_morphology_data_phase_1_and_2_-_2020JUNE25.csv'
# reformat data (even when there is duplicate)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, time

In [3]:
starting_dir = os.getcwd()
print ("starting_dir:" + str(starting_dir))

#complete_file_path = '/Users/kimd999/Dropbox/script/python/srpAnalytics/analysis/paritosh_original_then_edit/preprocessing/old_phase_I_II/344_zf_morphology_data_phase_1_and_2_-_2020JUNE25_tall.csv'
complete_file_path = '/Users/kimd999/research/projects/toxicity/data/Phase_1_and_2_for_PNNL_database_2020JUNE21/tall_344_zf_morphology_data_phase_1_and_2_-_2020JUNE25.csv'
morph_all_data = pd.read_csv(complete_file_path, header = 0)
pd.set_option('display.max_columns', None)

starting_dir:/Users/kimd999/Dropbox/script/python/srpAnalytics/analysis/paritosh_original_then_edit/preprocessing/old_phase_I_II


In [4]:
morph_all_data.head()

,chemical.id,bottle.id,conc,plate.id,well,date,endpoint,value
0,2752,TX000657,0.64,2,B03,2013,MO24,0.0
1,2752,TX000657,0.64,1,F03,2013,MO24,0.0
2,2752,TX000657,6.40,2,B08,2013,MO24,0.0
3,2752,TX000657,0.64,1,C03,2013,MO24,1.0
4,2752,TX000657,0.64,2,F03,2013,MO24,1.0


In [10]:
#new_morph_all_data = morph_all_data.loc[morph_all_data['chemical.id'] == 1532,['chemical.id', 'conc', 'plate.id', 'well', 'value']]
#new_morph_all_data.head()

new_morph_all_data = morph_all_data.loc[morph_all_data['chemical.id'] == 1532,['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']]
new_morph_all_data.head()

,chemical.id,conc,plate.id,well,endpoint,value
960,1532,64.0000,1,F01,MO24,0.0
961,1532,64.0000,1,G01,MO24,0.0
962,1532,0.0064,1,C05,MO24,0.0
963,1532,64.0000,1,G07,MO24,1.0
964,1532,0.0064,1,A11,MO24,0.0


In [12]:
morph_all_data_w_end = new_morph_all_data.loc[new_morph_all_data['endpoint'] == 'MO24',['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']]
morph_all_data_w_end.head()

,chemical.id,conc,plate.id,well,endpoint,value
960,1532,64.0000,1,F01,MO24,0.0
961,1532,64.0000,1,G01,MO24,0.0
962,1532,0.0064,1,C05,MO24,0.0
963,1532,64.0000,1,G07,MO24,1.0
964,1532,0.0064,1,A11,MO24,0.0


In [64]:
morph_all_data.tail()

,chemical.id,bottle.id,conc,plate.id,well,date,endpoint,value
1712635,3860,P518274,100.0,20329,H08,20200603,DNC_,0.0
1712636,3860,P518274,100.0,20329,H09,20200603,DNC_,0.0
1712637,3860,P518274,100.0,20329,H10,20200603,DNC_,0.0
1712638,3860,P518274,100.0,20329,H11,20200603,DNC_,0.0
1712639,3860,P518274,100.0,20329,H12,20200603,DNC_,0.0


In [65]:
np.sum(morph_all_data['value'] == 1)

53243

In [66]:
np.sum(morph_all_data['value'] == 0) # -> 82%

1404843

In [67]:
np.sum(np.isnan(morph_all_data['value']))

254554

In [68]:
morph_all_data.shape

(1712640, 8)

In [69]:
# Keep only relevant columns
columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
morph_all_data_select = morph_all_data.loc[:,columns_to_keep]
morph_all_data_select.head()

,chemical.id,conc,plate.id,well,endpoint,value
0,2752,0.64,2,B03,MO24,0.0
1,2752,0.64,1,F03,MO24,0.0
2,2752,6.40,2,B08,MO24,0.0
3,2752,0.64,1,C03,MO24,1.0
4,2752,0.64,2,F03,MO24,1.0


In [70]:
print ("len(morph_all_data_select):" + str(len(morph_all_data_select)))
morph_all_data_select = morph_all_data_select.drop_duplicates()
print ("len(morph_all_data_select):" + str(len(morph_all_data_select)))

len(morph_all_data_select):1712640
len(morph_all_data_select):1686144


In [79]:
start_time = time.time()

reformat_data = pd.DataFrame()
total_number_of_unique_chemicals = 0
total_number_of_chemical_plate_well = 0
for chemical_index in np.unique(morph_all_data['chemical.id']):
    total_number_of_unique_chemicals += 1
#    print ("\ntotal_number_of_unique_chemicals:" + str(total_number_of_unique_chemicals))

    #if ((chemical_index != 3760) and (chemical_index != 3859)):
    #    continue
    print("\nchemical_index:" + str(chemical_index))        

    morph_data_chemical = morph_all_data_select.loc[morph_all_data['chemical.id'] == chemical_index,:]

    # Append chemical_plate_well as a unique identifier
    morph_data_chemical.insert(0, 'chemical_plate_well', morph_data_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    print ("len(morph_data_chemical.chemical_plate_well):" + str(len(morph_data_chemical.chemical_plate_well)))    
    for cpw in np.unique(morph_data_chemical.chemical_plate_well):
        total_number_of_chemical_plate_well += 1
        temp_df = morph_data_chemical.loc[morph_data_chemical.chemical_plate_well == cpw,:]
        temp_df_grouped = temp_df.groupby(['chemical.id', 'plate.id', 'well'])
        #print ("temp_df_grouped:" + str(temp_df_grouped)) #<pandas.core.groupby.generic.DataFrameGroupBy object at 0x1292f0190>

        for name, group in temp_df_grouped:
#            if(len(group.endpoint) == 23):
                try:
                    temp = pd.DataFrame(
                        {
                        'chemical.id': np.unique(temp_df['chemical.id']),
                        'plate.id': np.unique(temp_df['plate.id']),
                        'well': np.unique(temp_df['well']),
                        'chemical_plate_well': np.unique(temp_df['chemical_plate_well']),
                        'conc': np.unique(temp_df['conc']),
                        'AXIS': temp_df.value[temp_df.endpoint == 'AXIS'].values,
                        'BRAI': temp_df.value[temp_df.endpoint == 'BRAI'].values,
                        'CFIN': temp_df.value[temp_df.endpoint == 'CFIN'].values,
                        'CIRC': temp_df.value[temp_df.endpoint == 'CIRC'].values,
                        'DNC_': temp_df.value[temp_df.endpoint == 'DNC_'].values,
                        'DP24': temp_df.value[temp_df.endpoint == 'DP24'].values,
                        'EYE_': temp_df.value[temp_df.endpoint == 'EYE_'].values,
                        'JAW_': temp_df.value[temp_df.endpoint == 'JAW_'].values,
                        'MO24': temp_df.value[temp_df.endpoint == 'MO24'].values,
                        'MORT': temp_df.value[temp_df.endpoint == 'MORT'].values,
                        'NC24': temp_df.value[temp_df.endpoint == 'NC24'].values,
                        'NC__': temp_df.value[temp_df.endpoint == 'NC__'].values,
                        'OTIC': temp_df.value[temp_df.endpoint == 'OTIC'].values,
                        'PE__': temp_df.value[temp_df.endpoint == 'PE__'].values,
                        'PFIN': temp_df.value[temp_df.endpoint == 'PFIN'].values,
                        'PIG_': temp_df.value[temp_df.endpoint == 'PIG_'].values,
                        'SM24': temp_df.value[temp_df.endpoint == 'SM24'].values,
                        'SNOU': temp_df.value[temp_df.endpoint == 'SNOU'].values,
                        'SOMI': temp_df.value[temp_df.endpoint == 'SOMI'].values,
                        'SWIM': temp_df.value[temp_df.endpoint == 'SWIM'].values,
                        'TRUN': temp_df.value[temp_df.endpoint == 'TRUN'].values,
                        'TR__': temp_df.value[temp_df.endpoint == 'TR__'].values,
                        'YSE_': temp_df.value[temp_df.endpoint == 'YSE_'].values,
                        }
                            )
                    reformat_data = pd.concat([reformat_data, temp])
                except:
                    print ("len(group.endpoint):" + str(len(group.endpoint)))
                    print ("chemical_plate_well:" + str(cpw))
                    # a=b
                
#            else:
 #               print ("len(group.endpoint):" + str(len(group.endpoint)))
  #              print ("chemical_plate_well:" + str(cpw))
#'''
print ("\ntotal_number_of_unique_chemicals:" + str(total_number_of_unique_chemicals))
print ("total_number_of_chemical_plate_well:" + str(total_number_of_chemical_plate_well))
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("Done, it took:"+str(time_took)) # took 15 minutes for 344 chemicals


chemical_index:53
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:54
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:56
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:57
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:58
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:60
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:61
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:62
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:63
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:64
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:65
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:66
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:67
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:69
len(morph_data_chemical.chemical_plate_well):4416

chemical_index:70
len(morph_data_

In [80]:
print (reformat_data.head())
print ("reformat_data.shape:" + str(reformat_data.shape))

   chemical.id  plate.id well chemical_plate_well  conc  AXIS  BRAI  CFIN  \
0           53      9289  A01         53_9289_A01  50.0   NaN   NaN   NaN   
0           53      9289  A02         53_9289_A02  35.6   0.0   0.0   0.0   
0           53      9289  A03         53_9289_A03  11.2   0.0   0.0   0.0   
0           53      9289  A04         53_9289_A04   5.0   1.0   0.0   0.0   
0           53      9289  A05         53_9289_A05   1.0   0.0   0.0   0.0   

   CIRC  DNC_  DP24  EYE_  JAW_  MO24  MORT  NC24  NC__  OTIC  PE__  PFIN  \
0   NaN   0.0   0.0   NaN   NaN   0.0   1.0   0.0   NaN   NaN   NaN   NaN   
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   1.0   
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   PIG_  SM24  SNOU  SOMI  SWIM  TRUN  TR__  YSE_  
0   NaN   0.0   NaN   

In [81]:
reformat_data_DNC_0 = pd.DataFrame()
reformat_data_DNC_0 = reformat_data.loc[reformat_data['DNC_'] == 0.0]
print ("reformat_data_DNC_0.shape:" + str(reformat_data_DNC_0.shape))

reformat_data_DNC_0.shape:(72784, 28)


In [82]:
reformat_data_DNC_empty = pd.DataFrame()
reformat_data_DNC_empty = reformat_data.loc[reformat_data['DNC_'] == '']
print ("reformat_data_DNC_empty.shape:" + str(reformat_data_DNC_empty.shape))

reformat_data_DNC_empty.shape:(0, 28)


In [83]:
#'''
nan_count = 0
zero_count = 0
one_count = 0
total_count = 0
for (columnName, columnData) in reformat_data.iteritems(): 
    if (columnName == "chemical.id") or (columnName == "plate.id") or (columnName == "well") or (columnName == "chemical_plate_well") or (columnName == "conc"):
        continue
#    print('Colunm Name : ', columnName) 
#    print('Column Contents : ', columnData.values[i])
#    print (len(columnData.values))
    for i in range(len(columnData.values)):
        total_count += 1
        if (str(columnData.values[i]) == "nan"):
            nan_count += 1
        elif (str(columnData.values[i]) == "0.0"):
            zero_count += 1
        elif (str(columnData.values[i]) == "1.0"):
            one_count += 1
#        print('Column Contents : ', columnData.values[i])
print ("nan_count:" + str(nan_count))
print ("zero_count:" + str(zero_count))
print ("one_count:" + str(one_count))
print ("total_count:" + str(total_count))
#'''

nan_count:251398
zero_count:1374391
one_count:52291
total_count:1678080


In [84]:
pd.set_option('display.max_columns', None)

In [85]:
reformat_data.to_csv('344_zf_morphology_data_phase_1_and_2_-_2020JUNE25_wide_DNC_all.csv',index=False)
reformat_data_DNC_0.to_csv('344_zf_morphology_data_phase_1_and_2_-_2020JUNE25_wide_DNC_0.csv',index=False)
reformat_data_DNC_empty.to_csv('344_zf_morphology_data_phase_1_and_2_-_2020JUNE25_wide_DNC_empty.csv',index=False)

In [86]:
len(np.unique(reformat_data['chemical.id']))

342

### Reformat behavioral data

In [ ]:
#complete_file_path = '/Users/pand381/Downloads/Phase 1 and 2 for PNNL database 2020JUNE21/344 zf LPR data phase 1 and 2 - 2020JUNE25.csv'
#behav_all_data = pd.read_csv(complete_file_path, header = 0)

In [ ]:
#behav_all_data.head()

In [ ]:
# Keep only relevant columns
#columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
#behav_all_data_select = behav_all_data.loc[:,columns_to_keep]
#behav_all_data_select.head()

In [ ]:
'''reformat_data = pd.DataFrame()

for chemical_index in np.unique(behav_all_data['chemical.id']):
    behav_data_chemical = behav_all_data_select.loc[behav_all_data['chemical.id'] == chemical_index,:]
    print("chemical_index:" + str(chemical_index))
    # Append chemical_plate_well as a unique identifier
    behav_data_chemical.insert(0, 'chemical_plate_well', behav_data_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    
    for cpw in np.unique(behav_data_chemical.chemical_plate_well):
        temp_df = behav_data_chemical.loc[behav_data_chemical.chemical_plate_well == cpw,:]
        temp_df_grouped = temp_df.groupby(['chemical.id', 'plate.id', 'well'])
        for name, group in temp_df_grouped:
            if(len(group.endpoint) == 15):
                temp = pd.DataFrame(
                        {
                        'chemical.id': np.unique(temp_df['chemical.id']),
                        'plate.id': np.unique(temp_df['plate.id']),
                        'well': np.unique(temp_df['well']),
                        'chemical_plate_well': np.unique(temp_df['chemical_plate_well']),
                        'conc': np.unique(temp_df['conc']),
                        't3': temp_df.value[temp_df.endpoint == 't3'].values,
                        't4': temp_df.value[temp_df.endpoint == 't4'].values,
                        't5': temp_df.value[temp_df.endpoint == 't5'].values,
                        't6': temp_df.value[temp_df.endpoint == 't6'].values,
                        't7': temp_df.value[temp_df.endpoint == 't7'].values,
                        't8': temp_df.value[temp_df.endpoint == 't8'].values,
                        't9': temp_df.value[temp_df.endpoint == 't9'].values,
                        't10': temp_df.value[temp_df.endpoint == 't10'].values,
                        't11': temp_df.value[temp_df.endpoint == 't11'].values,
                        't12': temp_df.value[temp_df.endpoint == 't12'].values,
                        't13': temp_df.value[temp_df.endpoint == 't13'].values,
                        't14': temp_df.value[temp_df.endpoint == 't14'].values,
                        't15': temp_df.value[temp_df.endpoint == 't15'].values,
                        't16': temp_df.value[temp_df.endpoint == 't16'].values,
                        't17': temp_df.value[temp_df.endpoint == 't17'].values,
                        }
                            )
                reformat_data = pd.concat([reformat_data, temp])'''

In [ ]:
#reformat_data.head()

In [ ]:
#reformat_data.shape

In [ ]:
#reformat_data.to_csv('Phase_I_II_t3_t17_LPR.csv',index=False)

### t0_t239 time points

In [ ]:
'''
reformat_data = pd.DataFrame()

#for chemical_index in np.unique([54]):
for chemical_index in np.unique(behav_all_data['chemical.id']):
    behav_data_chemical = behav_all_data_select.loc[behav_all_data['chemical.id'] == chemical_index,:]
    print(chemical_index)
    # Append chemical_plate_well as a unique identifier
    behav_data_chemical.insert(0, 'chemical_plate_well', behav_data_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    
    for cpw in np.unique(behav_data_chemical.chemical_plate_well):
        temp_df = behav_data_chemical.loc[behav_data_chemical.chemical_plate_well == cpw,:]
        #display(temp_df.head())
        temp_df_grouped = temp_df.groupby(['chemical.id', 'plate.id', 'well'])
        #display(temp_df_grouped.head())
        for name, group in temp_df_grouped:
            if(len(group.endpoint) == 240):
                temp = pd.DataFrame(
                        {
                        'chemical.id': np.unique(temp_df['chemical.id']),
                        'plate.id': np.unique(temp_df['plate.id']),
                        'well': np.unique(temp_df['well']),
                        'chemical_plate_well': np.unique(temp_df['chemical_plate_well']),
                        'conc': np.unique(temp_df['conc'])
                        })
                #print(temp.head())
                # Append additonal columns corresponding to time points
                max_time = 240
                for time_point in np.arange(max_time):
                    end_point = 't'+ str(time_point)
                    temp = pd.concat([temp, pd.DataFrame({end_point: temp_df.value[temp_df.endpoint == end_point].values})],axis = 1)
                #print(temp.head())
                #print(reformat_data)
                reformat_data = pd.concat([reformat_data, temp])
                #print(reformat_data)'''

In [ ]:
#display(reformat_data)

In [ ]:
#reformat_data.to_csv('Phase_I_II_t0_t239_LPR.csv',index=False)

In [ ]:
#len(np.unique(reformat_data['chemical.id']))

## Reformat EPR data

In [ ]:
#complete_file_path = '/Users/pand381/Downloads/Phase 1 and 2 for PNNL database 2020JUNE21/344 zf EPR data phase 1 and 2 - 2020JUNE25.csv'
#behav_all_data = pd.read_csv(complete_file_path, header = 0)

In [ ]:
#behav_all_data.head()

In [ ]:
# Keep only relevant columns
#columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
#behav_all_data_select = behav_all_data.loc[:,columns_to_keep]
#behav_all_data_select.head()

In [ ]:
'''
reformat_data = pd.DataFrame()

#for chemical_index in np.unique([54]):
for chemical_index in np.unique(behav_all_data['chemical.id']):
    behav_data_chemical = behav_all_data_select.loc[behav_all_data['chemical.id'] == chemical_index,:]
    print(chemical_index)
    # Append chemical_plate_well as a unique identifier
    behav_data_chemical.insert(0, 'chemical_plate_well', behav_data_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    
    for cpw in np.unique(behav_data_chemical.chemical_plate_well):
        temp_df = behav_data_chemical.loc[behav_data_chemical.chemical_plate_well == cpw,:]
        #display(temp_df.head())
        temp_df_grouped = temp_df.groupby(['chemical.id', 'plate.id', 'well'])
        #display(temp_df_grouped.head())
        for name, group in temp_df_grouped:
            if(len(group.endpoint) == 49):
                temp = pd.DataFrame(
                        {
                        'chemical.id': np.unique(temp_df['chemical.id']),
                        'plate.id': np.unique(temp_df['plate.id']),
                        'well': np.unique(temp_df['well']),
                        'chemical_plate_well': np.unique(temp_df['chemical_plate_well']),
                        'conc': np.unique(temp_df['conc'])
                        })
                #print(temp.head())
                # Append additonal columns corresponding to time points
                max_time = 49
                for time_point in np.arange(max_time):
                    end_point = 't'+ str(time_point+1)
                    temp = pd.concat([temp, pd.DataFrame({end_point: temp_df.value[temp_df.endpoint == end_point].values})],axis = 1)
                #print(temp.head())
                #print(reformat_data)
                reformat_data = pd.concat([reformat_data, temp])
                #print(reformat_data)
                '''

In [ ]:
#display(reformat_data)

In [ ]:
#reformat_data.to_csv('Phase_I_II_t1_t49_EPR.csv',index=False)